[![Kaggle link](https://img.shields.io/badge/изначальный%20датасет-Открыть-orange?style=social&logo=kaggle)](https://www.kaggle.com/datasets/leomartinelli/bullying-in-schools)


In [76]:
df_csv_path = '/Users/blacksnow/bmstu/ММО/Лабораторные работы/datasets/Bullying_2018_raw.csv'

# §1. Зависимости

In [77]:
import pandas as pd
import pandas.io.sql as psql
from tabulate import tabulate
import numpy as np
%matplotlib inline 

In [78]:
!python scripts/config.py

In [79]:
!python3 scripts/connect.py

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 15.2 (Debian 15.2-1.pgdg110+1) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)
Database connection closed.


## §2. Описание изначального датасета

In [80]:
bullying_df = psql.read_sql_query('SELECT * from bullying;',conn)
bullying_df.head()

/var/folders/g8/xtt465_57r53fy239yttrrrm0000gn/T/ipykernel_11725/3834999804.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,record,bullied_on_school_property_in_past_12_months,bullied_not_on_school_property_in_past_12_months,cyber_bullied_in_past_12_months,custom_age,sex,physically_attacked,physical_fighting,felt_lonely,close_friends,miss_school_no_permission,other_students_kind_and_helpful,parents_understand_problems,most_of_the_time_or_always_felt_lonely,missed_classes_or_school_without_permission,were_underweight,were_overweight,were_obese
0,1,Yes,Yes,None,13 years old,Female,0 times,0 times,Always,2,10 or more days,Never,Always,Yes,Yes,None,None,None
1,2,No,No,No,13 years old,Female,0 times,0 times,Never,3 or more,0 days,Sometimes,Always,No,No,None,None,None
2,3,No,No,No,14 years old,Male,0 times,0 times,Never,3 or more,0 days,Sometimes,Always,No,No,No,No,No
3,4,No,No,No,16 years old,Male,0 times,2 or 3 times,Never,3 or more,0 days,Sometimes,None,No,No,No,No,No
4,5,No,No,No,13 years old,Female,0 times,0 times,Rarely,3 or more,0 days,Most of the time,Most of the time,No,No,None,None,None


In [81]:
bullying_df.columns

Index(['record', 'bullied_on_school_property_in_past_12_months',
       'bullied_not_on_school_property_in_past_12_months',
       'cyber_bullied_in_past_12_months', 'custom_age', 'sex',
       'physically_attacked', 'physical_fighting', 'felt_lonely',
       'close_friends', 'miss_school_no_permission',
       'other_students_kind_and_helpful', 'parents_understand_problems',
       'most_of_the_time_or_always_felt_lonely',
       'missed_classes_or_school_without_permission', 'were_underweight',
       'were_overweight', 'were_obese'],
      dtype='object')

In [82]:
bull_desc = [
    ['Номер ответа','-'],
    ['В школе (за последние 12 месяцев)','Категориальный'], # - №2
    ['Вне школы (за последние 12 месяцев)','Категориальный'], # - №3
    ['Кибербуллинг (последние 12 месяцев?)','Категориальный'], # - №3
    ['Возраст респондента','Числовой'], # - №4
    ['Пол респондента','Категориальный'], # - №5
    ['Количество физических атак','Числовой'], # - №6
    ['Количество драк','Числовой'], # # - №7
    ['Ощущение одинокости','Категориальный'], # # - №8
    ['Количество близких друзей','Числовой'], # - №9
    ['Количество дней пропусков школы без разрешения','Числовой '], # - №10
    ['Доброта и отзывчивость окружающих','Категориальный'], # - №11
    ['Частота проблем взаимопонимания с родителями','Категориальный'], # - №12
    ['Ощущение одинокости большую части (частота)','Категориальный'], # # - №13
    ['Пропускали ли вы школу без разрешения (да/нет)','Категориальный'], # - №14
    ['Масса вашего тела в школе была ниже нормы?','Категориальный'], # - №15
    ['Масса вашего тела в школе была выше нормы?','Категориальный'], # - №16
    ['Вы страдали ожирением?','Категориальный'], # - №17
]


### §2.1 Статистика по датасету

In [83]:

def stats(data=bullying_df, desc=bull_desc, headers=('Номер','Название','Описание','Кат.','Числ.','Кол-во вариаций')):
    headers=('Номер','dtype','Описание','Тип','Кол-во пропусков','Кол-во вариаций')
    table = np.arange(data.shape[1]*len(headers))
    table = table.reshape(len(headers),data.shape[1])

    table = np.array([
                [x for x in range(1,19)],
                [d for d in data.dtypes],
                [a[0] for a in desc],
                [a[1] for a in desc],
                [data[col].isna().sum() for col in data.columns],
                [data[col].nunique() for col in data.columns]])

    table = table.transpose()

    print(
        tabulate(
            [data.shape],
            headers=('Кол-во строк', 'Кол-во столбцов'),
            tablefmt="github"),
        end='\n\n\n'
        )
    
    print(
        tabulate(
            table,
            headers=headers,
            tablefmt='github'
        ),
        end='\n\n\n'
        )

stats()

|   Кол-во строк |   Кол-во столбцов |
|----------------|-------------------|
|          56981 |                18 |


|   Номер | dtype   | Описание                                       | Тип            |   Кол-во пропусков |   Кол-во вариаций |
|---------|---------|------------------------------------------------|----------------|--------------------|-------------------|
|       1 | int64   | Номер ответа                                   | -              |                  0 |             56981 |
|       2 | object  | В школе (за последние 12 месяцев)              | Категориальный |               1239 |                 2 |
|       3 | object  | Вне школы (за последние 12 месяцев)            | Категориальный |                489 |                 2 |
|       4 | object  | Кибербуллинг (последние 12 месяцев?)           | Категориальный |                571 |                 2 |
|       5 | object  | Возраст респондента                            | Числовой       |                108 

### §2.2. Пропуски изначального датасета

In [84]:
df_variants = []

for col in bullying_df.columns:
    print(f"Варианты колонки: {col}: ", end='')
    variants = bullying_df[col].unique()
    print(variants)

Варианты колонки: record: [    1     2     3 ... 57093 57094 57095]
Варианты колонки: bullied_on_school_property_in_past_12_months: ['Yes' 'No' None]
Варианты колонки: bullied_not_on_school_property_in_past_12_months: ['Yes' 'No' None]
Варианты колонки: cyber_bullied_in_past_12_months: [None 'No' 'Yes']
Варианты колонки: custom_age: ['13 years old' '14 years old' '16 years old' '12 years old'
 '15 years old' '11 years old or younger' '17 years old' None
 '18 years old or older']
Варианты колонки: sex: ['Female' 'Male' None]
Варианты колонки: physically_attacked: ['0 times' '1 time' '12 or more times' '4 or 5 times' '2 or 3 times'
 '10 or 11 times' '8 or 9 times' '6 or 7 times' None]
Варианты колонки: physical_fighting: ['0 times' '2 or 3 times' '1 time' '4 or 5 times' '6 or 7 times'
 '8 or 9 times' '10 or 11 times' None '12 or more times']
Варианты колонки: felt_lonely: ['Always' 'Never' 'Rarely' 'Sometimes' 'Most of the time' None]
Варианты колонки: close_friends: ['2' '3 or more' '0'

### 2.3 В итоге 

Датасет содержит большое количество пропусков, что видно из пункта 2.2. 

In [130]:
import plotly.express as px

bullying_df_gaps_count = bullying_df.isna().sum()
bullying_df_gaps_count.head()

fig = px.bar(bullying_df_gaps_count, orientation='h',range_x=[0,56000],text_auto='.2s')

fig.show()

## §3. Datenluecken Loeschung

Удалим все пропуски в данных с помощью

In [85]:
def normalized_source(df=bullying_df):
    return df

stats(data=normalized_source())


|   Кол-во строк |   Кол-во столбцов |
|----------------|-------------------|
|          56981 |                18 |


|   Номер | dtype   | Описание                                       | Тип            |   Кол-во пропусков |   Кол-во вариаций |
|---------|---------|------------------------------------------------|----------------|--------------------|-------------------|
|       1 | int64   | Номер ответа                                   | -              |                  0 |             56981 |
|       2 | object  | В школе (за последние 12 месяцев)              | Категориальный |               1239 |                 2 |
|       3 | object  | Вне школы (за последние 12 месяцев)            | Категориальный |                489 |                 2 |
|       4 | object  | Кибербуллинг (последние 12 месяцев?)           | Категориальный |                571 |                 2 |
|       5 | object  | Возраст респондента                            | Числовой       |                108 